In [1]:
import torch
from transformers import WhisperModel, WhisperFeatureExtractor
import torchaudio
import iree.turbine.aot as aot

/home/gilro/ml_stuff/infra/iree/venv_online_everything/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = WhisperModel.from_pretrained("openai/whisper-tiny")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")

In [3]:
samples, sr = torchaudio.load("../data/gilro_test.wav")
input_features = feature_extractor(
    samples[0][:], sampling_rate=sr, return_tensors="pt"
).input_features
print(input_features.shape)
decoder_input_ids = torch.tensor([[1]]) * model.config.decoder_start_token_id
model(
    input_features,
    decoder_input_ids=decoder_input_ids,
    use_cache=False,
    decoder_attention_mask=torch.tensor([[[[True]]]]),
).last_hidden_state.shape

torch.Size([1, 80, 3000])


/home/gilro/ml_stuff/infra/iree/venv_online_everything/lib/python3.12/site-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/home/gilro/ml_stuff/infra/iree/venv_online_everything/lib/python3.12/site-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Pl

torch.Size([1, 1, 384])

In [5]:
class Wrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        with torch.no_grad():
            return self.model.forward(
                x,
                decoder_input_ids=torch.tensor([[1]])
                * self.model.config.decoder_start_token_id,
                use_cache=False,
                decoder_attention_mask=torch.tensor([[[[True]]]]),
            ).last_hidden_state


model_ = Wrapper(model).cpu().eval()
ref = model_.forward(input_features)

export_output = aot.export(model_, input_features)
export_output.save_mlir("whisper_tiny.mlir")

In [ ]:
ref = model_.forward(input_features)

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

In [17]:
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

model.config.forced_decoder_ids = None

In [22]:
samples, sr = torchaudio.load("../data/gilro_test.wav")
input_ids = processor(
    samples[0][:], sampling_rate=sr, return_tensors="pt"
).input_features
predicted_ids = model.generate(input_ids)
transcription = processor.batch_decode(predicted_ids)
transcription

[' Hello, how are you? Are you here? Can you understand me?']

In [6]:
input_ids

tensor([[[ 1.5989,  1.5989,  1.5989,  ..., -0.4001, -0.4001, -0.4001],
         [ 1.3247,  1.3247,  1.3247,  ..., -0.4001, -0.4001, -0.4001],
         [-0.4001, -0.4001, -0.4001,  ..., -0.4001, -0.4001, -0.4001],
         ...,
         [-0.4001, -0.4001, -0.4001,  ..., -0.4001, -0.4001, -0.4001],
         [-0.4001, -0.4001, -0.4001,  ..., -0.4001, -0.4001, -0.4001],
         [-0.4001, -0.4001, -0.4001,  ..., -0.4001, -0.4001, -0.4001]]])

In [ ]:
model = model.eval()


class Wrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):

        return self.model.forward(
            x,
            decoder_input_ids=torch.tensor([[1]])
            * self.model.config.decoder_start_token_id,
        )
        # return self.model.model(
        #     x,
        # )


model_ = Wrapper(model).cpu().eval()

# export_output = aot.export(model_, input_ids)


# model_(torch.randn((1,80,3000)))
model_.forward(input_ids)
# input_ids.shape

ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds

In [ ]:
class NothingModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(100, 13)

    def forward(self, x):
        return torch.nn.ReLU()(self.fc(x))


model = NothingModel()
sample_inputs = torch.randn(100)
x = model(sample_inputs)

export_output = aot.export(model, sample_inputs)
export_output.save_mlir("converted_nothing_model.mlir")

Signature: export_output.save_mlir(file_path: Union[str, pathlib.Path])
Docstring:
Saves the current compilation IR to a path on disk.

Args:
    file_path: Path to save the file. If it has a ".mlirbc"
      extension, it will be saved as bytecode. Otherwise as
      text.
File:      ~/ml_stuff/infra/iree/venv_online_everything/lib/python3.12/site-packages/iree/turbine/aot/exporter.py
Type:      method